In [11]:
import os
import cv2
import numpy as np
import os
from PIL import Image
import random

In [12]:
def recortar_redimensionar_imagen(input_path, output_path, nuevo_tamano):
    # Abrir la imagen
    imagen = Image.open(input_path)

    # Obtener el tamaño actual de la imagen
    ancho_actual, alto_actual = imagen.size

    # Calcular las coordenadas para recortar centrado
    izquierda = (ancho_actual - nuevo_tamano[0]) // 2 + 10
    arriba = (alto_actual - nuevo_tamano[1]) // 2 + 10
    derecha = (ancho_actual + nuevo_tamano[0]) // 2 + 10
    abajo = (alto_actual + nuevo_tamano[1]) // 2 + 10

    # Recortar la imagen
    imagen_recortada = imagen.crop((izquierda, arriba, derecha, abajo))

    # Redimensionar la imagen
    imagen_redimensionada = imagen_recortada.resize(nuevo_tamano, Image.ANTIALIAS)

    # Guardar la imagen resultante
    imagen_redimensionada.save(output_path)

def batch_process_images(input_folder, output_folder, nuevo_tamano):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # List all files in the input folder
    files = os.listdir(input_folder)

    # Process each image in the folder
    for file in files:
        if file.endswith(".png") or file.endswith(".jpg") or file.endswith(".jpeg"):
            # Construct full paths for input and output
            input_path = os.path.join(input_folder, file)
            output_path = os.path.join(output_folder, file)

            # Call the function to process the image
            recortar_redimensionar_imagen(input_path, output_path, nuevo_tamano)

if __name__ == "__main__":
    # Set the input folder, output folder, and nuevo_tamano
    input_folder = "6-final"
    output_folder = "7-final"
    nuevo_tamano = (2250, 1830)

    # Call the batch processing function
    batch_process_images(input_folder, output_folder, nuevo_tamano)

C:\Users\Javier\AppData\Local\Temp\ipykernel_14392\2519443479.py:18: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  imagen_redimensionada = imagen_recortada.resize(nuevo_tamano, Image.ANTIALIAS)


In [13]:
def crecimiento_regiones(imagen, semillas, umbral):
    # Inicializar la máscara con ceros
    mascara = np.zeros_like(imagen)

    # Establecer las semillas iniciales
    seeds = semillas.copy()

    while seeds:
        # Obtener la posición de la semilla
        seed = seeds.pop(0)
        x, y = seed

        # Comprobar si el píxel está dentro de los límites de la imagen
        if 0 <= x < imagen.shape[0] and 0 <= y < imagen.shape[1]:
            # Comprobar si el píxel no ha sido visitado
            if mascara[x, y] == 0:
                # Comprobar si la diferencia de intensidad es menor que el umbral
                if all(abs(int(imagen[x, y]) - int(imagen[sem])) < umbral for sem in semillas):
                    # Agregar el píxel a la región y la semilla
                    mascara[x, y] = 255
                    seeds.append((x + 1, y))
                    seeds.append((x - 1, y))
                    seeds.append((x, y + 1))
                    seeds.append((x, y - 1))

    return mascara

In [14]:
output_folder = '8-final/'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

def dividir_y_aplicar_algoritmo(imagen_path, columnas, filas, algoritmo):
    imagen = cv2.imread(imagen_path)
    imagen_gris = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
    alto_total, ancho_total = imagen_gris.shape
    ancho_trozo = ancho_total // columnas
    alto_trozo = alto_total // filas
    i = 0
    for fila in range(filas):
        for columna in range(columnas):
            izquierda = columna * ancho_trozo
            arriba = fila * alto_trozo
            derecha = (columna + 1) * ancho_trozo
            abajo = (fila + 1) * alto_trozo
            trozo = imagen_gris[arriba:abajo, izquierda:derecha]
            esquina_superior_izquierda = (0, 0)
            esquina_superior_derecha = (0, trozo.shape[1] - 1)
            esquina_inferior_izquierda = (trozo.shape[0] - 1, 0)
            esquina_inferior_derecha = (trozo.shape[0] - 1, trozo.shape[1] - 1)
            lista_semillas = [esquina_superior_izquierda, esquina_superior_derecha, esquina_inferior_izquierda, esquina_inferior_derecha]
            umbral_crecimiento = 99
            trozo = crecimiento_regiones(trozo, lista_semillas, umbral_crecimiento)
            i += 1
            imagen_gris[arriba:abajo, izquierda:derecha] = trozo

    output_path = os.path.join(output_folder, f'{os.path.basename(imagen_path)}')
    cv2.imwrite(output_path, imagen_gris)

def batch_process_images(input_folder, columnas, filas, algoritmo):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    files = os.listdir(input_folder)

    for file in files:
        if file.endswith(".png") or file.endswith(".jpg") or file.endswith(".jpeg"):
            input_path = os.path.join(input_folder, file)
            dividir_y_aplicar_algoritmo(input_path, columnas, filas, algoritmo)

if __name__ == "__main__":
    input_folder = "7-final"
    columnas = 5
    filas = 4

    batch_process_images(input_folder, columnas, filas, crecimiento_regiones)

In [15]:
output_folder = '9-final/'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

def aplicar_colores_a_contornos(imagen_path, area_minima):
    # Cargar la imagen a color
    imagen = cv2.imread(imagen_path)

    # Convertir la imagen a escala de grises
    imagen_gris = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)

    # Invertir la imagen en escala de grises
    imagen_invertida = cv2.bitwise_not(imagen_gris)

    # Encontrar contornos en la imagen invertida
    contornos, _ = cv2.findContours(imagen_invertida, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Crear una nueva imagen en color
    nueva_imagen = np.zeros_like(imagen)

    # Iterar sobre cada contorno y asignar un color aleatorio
    for contorno in contornos:
        area_contorno = cv2.contourArea(contorno)

        # Filtrar por área mínima
        if area_contorno > area_minima:
            color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
            cv2.drawContours(nueva_imagen, [contorno], -1, color, thickness=cv2.FILLED)

    output_path = os.path.join(output_folder, f'{os.path.basename(imagen_path)}')

    # Guardar la nueva imagen en color
    cv2.imwrite(output_path, nueva_imagen)

def batch_process_images(input_folder, area_minima):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    files = os.listdir(input_folder)

    for file in files:
        if file.endswith(".png") or file.endswith(".jpg") or file.endswith(".jpeg"):
            input_path = os.path.join(input_folder, file)
            aplicar_colores_a_contornos(input_path, area_minima)

if __name__ == "__main__":
    input_folder = "8-final"
    area_minima = 100

    batch_process_images(input_folder, area_minima)